In [34]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1)
plt.style.use('ggplot')

import pandas as pd
import numpy as np
import random as rnd

from sklearn.cross_validation import KFold, cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train=pd.read_csv('../trainval.csv')
#validate=pd.read_csv('../validate.csv')
#test=pd.read_csv('../testwDSM.csv')

train.shape, validate.shape, #test.shape

((1508933, 48), (743207, 48))

In [35]:
#train.describe()
train = train.sample(10000)
validate = validate.sample(6000)
train.shape, validate.shape, validate.head(2)

((10000, 48),
 (6000, 48),
         YEAR  AGE  GENDER  RACE  ETHNIC  MARSTAT  EDUC  EMPLOY  DETNLF  PREG  \
 228676  2013    6       1     5       5        2     3       3       0     2   
 445469  2011   10       1     5       5        4     3       4       5     2   
 
          ...     BENZFLG  TRNQFLG  BARBFLG  SEDHPFLG  INHFLG  OTCFLG  \
 228676   ...           0        0        0         0       0       0   
 445469   ...           0        0        0         0       0       0   
 
         OTHERFLG  ALCDRUG  DSMCRIT  PSYPROB  
 228676         0        3        8        2  
 445469         0        2        8        2  
 
 [2 rows x 48 columns])

In [36]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['DSMCRIT', 'YEAR', 'STFIPS', 
             'ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'METHFLG', 'OPSYNFLG', 'PCPFLG', 'HALLFLG', 'MTHAMFLG', 
             'AMPHFLG', 'STIMFLG', 'BENZFLG', 'TRNQFLG', 'BARBFLG', 'SEDHPFLG', 'INHFLG', 'OTCFLG', 'OTHERFLG', 
             'ALCDRUG', #'NUMSUBS'
             ]

X_train = train.drop(drop_list, axis=1)
Y_train = train["DSMCRIT"]
X_validate = validate.drop(drop_list, axis=1)
Y_validate = validate["DSMCRIT"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, X_validate.shape, #X_test.shape



((10000, 26), (6000, 26))

In [37]:
print X_train.columns.tolist()

['AGE', 'GENDER', 'RACE', 'ETHNIC', 'MARSTAT', 'EDUC', 'EMPLOY', 'DETNLF', 'PREG', 'VET', 'LIVARAG', 'ARRESTS', 'SERVSETA', 'METHUSE', 'PSOURCE', 'DETCRIM', 'NOPRIOR', 'SUB1', 'ROUTE1', 'FREQ1', 'FRSTUSE1', 'SUB2', 'SUB3', 'NUMSUBS', 'IDU', 'PSYPROB']


In [38]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
#onehotlabels.shape
X_train = onehotlabels

onehotlabels = enc.transform(X_validate).toarray()
X_validate = onehotlabels

X_train.shape, X_validate.shape

((10000, 174), (6000, 174))

In [39]:
# Logistic Regression
logreg = LogisticRegression(n_jobs=-1)
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)

l_acc_log = cross_val_score(logreg, X_train, Y_train, cv=5)
acc_log = round(np.mean(l_acc_log), 3)

l_acc_log = ['%.3f' % elem for elem in l_acc_log]
print l_acc_log
print acc_log

['0.681', '0.671', '0.676', '0.671', '0.684']
0.677


In [40]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=200, max_depth=20, n_jobs=-1)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)

l_acc_random_forest = cross_val_score(random_forest, X_train, Y_train, cv=5)
acc_random_forest = round(np.mean(l_acc_random_forest), 3)

l_acc_random_forest = ['%.3f' % elem for elem in l_acc_random_forest]
print l_acc_random_forest
print acc_random_forest

['0.684', '0.673', '0.682', '0.677', '0.688']
0.681


In [41]:
# Linear SVC
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
#Y_pred = linear_svc.predict(X_test)

l_acc_linear_svc = cross_val_score(linear_svc, X_train, Y_train, cv=5)
acc_linear_svc = round(np.mean(l_acc_linear_svc), 3)
l_acc_linear_svc = ['%.3f' % elem for elem in l_acc_linear_svc]
print l_acc_linear_svc
print acc_linear_svc

['0.671', '0.662', '0.669', '0.668', '0.671']
0.668


In [42]:
print 'predict-sub2-woflags-newsplit-sample20000'
models = pd.DataFrame({
    'Model': ['Logistic Regression', 
              'Random Forest','Linear SVC'],
    'Cross Validation': [l_acc_log, 
              l_acc_random_forest, l_acc_linear_svc],
    'Cross Validation Mean': [acc_log, 
              acc_random_forest, acc_linear_svc]
    })
models.sort_values(by='Cross Validation Mean', ascending=False)


predict-sub2-woflags-newsplit-sample20000


,Cross Validation,Cross Validation Mean,Model
1,"[0.684, 0.673, 0.682, 0.677, 0.688]",0.681,Random Forest
0,"[0.681, 0.671, 0.676, 0.671, 0.684]",0.677,Logistic Regression
2,"[0.671, 0.662, 0.669, 0.668, 0.671]",0.668,Linear SVC
